# Imports

In [1]:
from abc import ABC, abstractmethod
from datetime import datetime, date
from typing import Union, Tuple
import calendar
import math

# Day Count Conventions

Day count conventions are methods used to calculate the number of days between two dates for the purpose of calculating interest payments or other calculations. Different markets and instruments use different conventions, which can lead to slightly different results.

## Overview Table

| Convention | Description | Common Usage | Formula |
|------------|-------------|--------------|---------|
| Actual/360 | Uses actual number of days with 360-day year | USD LIBOR, Money Markets | $\frac{\text{Actual Days}}{360}$ |
| Actual/365 Fixed | Uses actual number of days with 365-day year | GBP LIBOR, Sterling Markets | $\frac{\text{Actual Days}}{365}$ |
| Actual/Actual ISDA | Uses actual days and actual days in year | Bonds, Swaps | $\sum\frac{\text{Days in Period}}{\text{Days in Year}}$ |
| 30/360 US | Assumes 30-day months in 360-day year | US Corporate Bonds | $\frac{360(Y_2-Y_1) + 30(M_2-M_1) + (D_2-D_1)}{360}$ |
| 30E/360 | European version of 30/360 | European Bonds | $\frac{360(Y_2-Y_1) + 30(M_2-M_1) + (D_2-D_1)}{360}$ |
| 30E/360 ISDA | ISDA modified version of 30E/360 | Derivatives | $\frac{360(Y_2-Y_1) + 30(M_2-M_1) + (D_2-D_1)}{360}$ |
| Actual/365L | Leap year adjusted version | GBP Markets | $\frac{\text{Actual Days}}{365 \text{ or } 366}$ |

## Detailed Explanations

### 1. Actual/360 (French)
- Also known as: Act/360, A/360, French
- **Calculation**: Actual number of days in period divided by 360
- **Usage**:
  - USD LIBOR
  - EUR deposits
  - Interest rate swaps in EUR, USD, and CHF
- **Example**:
  ```python
  year_fraction = actual_days / 360
  ```

### 2. Actual/365 Fixed (English)
- Also known as: Act/365F, A/365F, English
- **Calculation**: Actual number of days in period divided by 365
- **Usage**:
  - GBP LIBOR
  - GBP interest rate swaps
  - Australian and Hong Kong markets
- **Note**: Ignores leap years
- **Example**:
  ```python
  year_fraction = actual_days / 365
  ```

### 3. Actual/Actual ISDA
- **Calculation**: Split calculation by calendar year
- **Usage**:
  - Fixed rate bonds
  - Interest rate swaps
  - Forward rate agreements
- **Method**:
  1. Split the period into portions falling in each calendar year
  2. For each portion, divide by days in that specific year (365 or 366)
  3. Sum the results
- **Example**:
  ```python
  year_fraction = days_in_year1/days_in_full_year1 + days_in_year2/days_in_full_year2
  ```

### 4. 30/360 US (Bond Basis)
- **Calculation**: Assumes 30-day months
- **Rules**:
  1. If D1 is 31, change to 30
  2. If D2 is 31 and D1 is 30 or 31, change D2 to 30
- **Usage**:
  - US corporate bonds
  - US municipal bonds
  - US agency bonds
- **Formula**:
  ```
  Year Fraction = [360(Y2-Y1) + 30(M2-M1) + (D2-D1)] / 360
  ```

### 5. 30E/360 (European)
- Also known as: 30/360 ISMA, 30/360 European, Eurobond Basis
- **Rules**:
  1. If D1 is 31, change to 30
  2. If D2 is 31, change to 30
- **Usage**:
  - Eurobonds
  - European government bonds
- **Formula**: Same as 30/360 but with different end-of-month rules

### 6. 30E/360 ISDA
- **Additional Rules**:
  1. If date is last day of February, change to 30
  2. Special handling for maturity dates
- **Usage**:
  - ISDA derivatives
  - Credit default swaps
- **Note**: Most precise version of 30/360 methods

### 7. Actual/365L
- **Calculation**: Actual days divided by 365 (or 366 for leap year periods)
- **Special Feature**: Adjusts denominator for leap years
- **Usage**: Primarily in GBP markets
- **Rules**:
  1. Use 366 if period includes February 29
  2. Use 365 otherwise

## Example Comparison

For a period from February 28, 2024 to February 28, 2025:

```python
# Sample year fractions:
Actual/360:        1.013889
Actual/365 Fixed:  1.000000
Actual/Actual:     1.000000
30/360 US:         1.000000
30E/360:           1.000000
```

## Market Standard Uses

| Market/Instrument | Typical Convention |
|------------------|-------------------|
| USD Money Markets | Actual/360 |
| GBP Money Markets | Actual/365 Fixed |
| EUR Money Markets | Actual/360 |
| US Corporate Bonds | 30/360 US |
| European Bonds | 30E/360 |
| Interest Rate Swaps | Actual/Actual ISDA |
| Cross Currency Swaps | Various (currency dependent) |

## References

1. ISDA Definitions (2006)
2. Bond Market Association Guidelines
3. ICMA Rule Book & Recommendations

In [2]:
class DayCountConvention(ABC):
    """
    Abstract base class for day count conventions.
    Each specific convention inherits from this and implements its own calculation logic.
    """
    @abstractmethod
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        """Calculate the year fraction between two dates"""
        pass
    
    def _to_date(self, dt: Union[date, datetime]) -> date:
        """Convert datetime to date if necessary"""
        return dt.date() if isinstance(dt, datetime) else dt
    
    def _validate_dates(self, start_date: date, end_date: date) -> None:
        """Validate that end date is after start date"""
        if end_date < start_date:
            raise ValueError("End date must be after start date")

In [3]:
class Actual360(DayCountConvention):
    """
    Actual/360 day count convention.
    Also known as Act/360, A/360, French.
    
    Used for:
    - USD LIBOR
    - EUR deposits
    - Interest rate swaps in EUR, USD, and CHF
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
        return (end - start).days / 360

class Actual365Fixed(DayCountConvention):
    """
    Actual/365 Fixed day count convention.
    Also known as Act/365F, A/365F, English.
    
    Used for:
    - GBP LIBOR
    - GBP interest rate swaps
    - AUD and HKD markets
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
        return (end - start).days / 365

class Actual365L(DayCountConvention):
    """
    Actual/365L day count convention.
    Used mainly for GBP capital markets and leap year adjustments.
    
    Special handling for leap years in the calculation period.
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
        # Check if February 29 falls within the period
        def has_feb29(year: int) -> bool:
            return calendar.isleap(year)
        years_between = range(start.year, end.year + 1)
        leap_days = sum(1 for year in years_between if has_feb29(year))
        
        # If period includes Feb 29, use 366, else use 365
        if leap_days > 0:
            denominator = 366
        else:
            denominator = 365
            
        return (end - start).days / denominator

class Thirty360US(DayCountConvention):
    """
    30/360 US day count convention.
    Also known as 30/360 Bond Basis.
    
    Used for:
    - US corporate bonds
    - US municipal bonds
    - US agency bonds
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
        # Get year, month, day
        y1, m1, d1 = start.year, start.month, start.day
        y2, m2, d2 = end.year, end.month, end.day
        
        # US 30/360 rules
        if d1 == 31:
            d1 = 30
        if d2 == 31 and d1 == 30:
            d2 = 30
            
        return (360 * (y2 - y1) + 30 * (m2 - m1) + (d2 - d1)) / 360

class Thirty360European(DayCountConvention):
    """
    30E/360 day count convention.
    Also known as 30/360 Eurobond Basis, 30/360 ISMA, 30/360 European.
    
    Used for:
    - Eurobonds
    - European government bonds
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
                # Get year, month, day

        y1, m1, d1 = start.year, start.month, start.day
        y2, m2, d2 = end.year, end.month, end.day
        
        # European 30/360 rules
        if d1 == 31:
            d1 = 30
        if d2 == 31:
            d2 = 30
            
        return (360 * (y2 - y1) + 30 * (m2 - m1) + (d2 - d1)) / 360

class Thirty360ISDA(DayCountConvention):
    """
    30E/360 ISDA day count convention.
    Special version of 30E/360 specified by ISDA.
    
    Used for:
    - Interest rate swaps under ISDA agreements
    - Credit default swaps
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
        # Get year, month, day
        y1, m1, d1 = start.year, start.month, start.day
        y2, m2, d2 = end.year, end.month, end.day
        
        # ISDA 30/360 rules
        if d1 == 31:
            d1 = 30
        if d2 == 31:
            d2 = 30
            
        # Additional ISDA rule for end of month
        if m1 == 2 and d1 == calendar.monthrange(y1, 2)[1]:  # Last day of February
            d1 = 30
        if m2 == 2 and d2 == calendar.monthrange(y2, 2)[1]:  # Last day of February
            d2 = 30
            
        return (360 * (y2 - y1) + 30 * (m2 - m1) + (d2 - d1)) / 360

class ActualActualISDA(DayCountConvention):
    """
    Actual/Actual ISDA day count convention.
    
    Used for:
    - Fixed rate bonds
    - Interest rate swaps
    - Forward rate agreements
    """
    def year_fraction(
        self,
        start_date: Union[date, datetime],
        end_date: Union[date, datetime]
    ) -> float:
        # Transform to dates and validate
        start = self._to_date(start_date)
        end = self._to_date(end_date)
        self._validate_dates(start, end)
        
        # Calculate actual days / actual days in a year, handling leap years
        total_years = 0
        current_date = start
        while current_date < end:
            year_end = date(current_date.year, 12, 31)
            if year_end > end:
                year_end = end
                
            days_in_year = 366 if calendar.isleap(current_date.year) else 365
            year_fraction = (year_end - current_date).days / days_in_year
            total_years += year_fraction
            current_date = date(year_end.year + 1, 1, 1)
        
        return total_years